# High dimensional bilinear game on the simplex

Results on the following constrained bilinear game, where $\mathbf{x}_1, \mathbf{x}_2 \in \mathbb{R}^{500}$, and $\eta \in (0,1)$:

$ 
min_{\mathbf{x}_1} max_{\mathbf{x}_2} \quad \eta \cdot \mathbf{x}_1^\intercal + (1-\eta) \cdot \mathbf{x}_1^\intercal \mathbf{x}_2 - \eta \cdot \mathbf{x}_2^\intercal \mathbf{x}_2
$

$
\hspace{2em} \text{s.t.} \qquad \mathbf{x} \geq \mathbf{0}
$

$
\hspace{5em} \mathbf{e}^\intercal \mathbf{x}_1 = 1
$

$
\hspace{5em} \mathbf{e}^\intercal \mathbf{x}_2 = 1
$

# Imports

In [11]:
import numpy as np
import time
import scipy.optimize as opt
from scipy import linalg
import matplotlib.pyplot as plt
from cvxopt import matrix,solvers
solvers.options['show_progress'] = False

# Experiments on conditioning

## I-ACVI

In [ ]:
import numpy as np
import time 

eps = .02  # target relative error
dim = 500  # dim(x1) == dim(x2) == dim

for conditioning in range(1,11):

  beta, mu, delta, K, lr, T = .5, 1e-5, .5, 50, .003, 200  
  if conditioning in [1,2,3]:
    l = 20
  elif conditioning in [4,5,6]:
    l = 50
  else:
    l = 100

  # Building HBG-v2 matrices
  eigs = np.linspace(1, conditioning, dim)
  A1 = np.concatenate((np.zeros((dim, dim)), np.diag(eigs)), axis=1)
  A2 = np.concatenate((-np.diag(eigs), np.zeros((dim, dim))), axis=1)
  A  = np.concatenate((A1, A2), axis=0)
  x_opt = 1/eigs
  x_opt /= x_opt.sum()
  x_opt = np.concatenate((x_opt, x_opt), axis=0).reshape(-1, 1)

  # Build projection matrix Pc
  temp1 = np.concatenate((np.ones((1,dim)), np.zeros((1,dim))), axis=1)
  temp2 = np.concatenate((np.zeros((1,dim)), np.ones((1,dim))), axis=1)
  C = np.concatenate((temp1,temp2), axis=0)
  d = np.ones((2,1))
  temp = np.linalg.inv(np.dot(C, C.T))
  temp = np.dot(C.T, temp)
  dc = np.dot(temp, d)
  Pc = np.identity(2*dim) - np.dot(temp,C)

  np.random.seed(0)

  # Initialize players
  init = np.random.rand(2*dim, 1)
  init[:dim] = init[:dim] / np.sum(init[:dim]) # ensuring it is part of the simplex
  init[dim:] = init[dim:] / np.sum(init[dim:])  
  z_x = np.copy(init)
  z_y = np.copy(init)
  z_lmd = np.zeros(init.shape)

  finished, cnt, t0 = False, 0, time.time()
    
  for _ in range(T):
    mu *= delta
    
    for _ in range(K):
      cnt += 1
      # Solve approximately the X problem (line 8 of algorithm)
      for _ in range(l):
        g = z_x + 1/beta * np.dot(Pc, np.dot(A,z_x)) - np.dot(Pc, z_y) + 1/beta * np.dot(Pc, z_lmd) - dc
        z_x -= lr * g

      if np.linalg.norm(z_x-x_opt)/np.linalg.norm(x_opt) <= eps:
        finished = True
        print(f"Reached a relative error of {eps} after {cnt} iterations in {time.time()-t0:.2f} sec.")
        break

      # Solve approximately the Y problem (line 9 of algorithm)
      for _ in range(l): 
        assert all(z_y > 0) # ensuring the log terms are positive
        g = - mu * 1/z_y + beta*(z_y - z_x - z_lmd/beta)
        z_y -= lr * g
        
      # Update lambdas (line 10 of algorithm)
      z_lmd += beta*(z_x-z_y)

    if finished:
      break

## EG

In [ ]:
import numpy as np
import time 
from cvxopt import matrix,solvers
solvers.options['show_progress'] = False

eps = .02  # target relative error
dim = 500  # dim(x1) == dim(x2) == dim
max_time = 700

lr = 0.3

for conditioning in range(1,11):

  lr *= 0.9   
  T = 10000    

  # Building HBG matrices
  eta = 0.0
  eigs = np.linspace(1, conditioning, dim)
  A1 = np.concatenate((np.zeros((dim, dim)), np.diag(eigs)), axis=1)
  A2 = np.concatenate((-np.diag(eigs), np.zeros((dim, dim))), axis=1)
  A  = np.concatenate((A1, A2), axis=0)
  x_opt = 1/eigs
  x_opt /= x_opt.sum()
  x_opt = np.concatenate((x_opt, x_opt), axis=0).reshape(-1, 1)

  # Build projection matrix Pc
  temp1 = np.concatenate((np.ones((1,dim)), np.zeros((1,dim))), axis=1)
  temp2 = np.concatenate((np.zeros((1,dim)), np.ones((1,dim))), axis=1)
  C = np.concatenate((temp1,temp2), axis=0)
  d = np.ones((2,1))
  temp = np.linalg.inv(np.dot(C, C.T))
  temp = np.dot(C.T, temp)
  dc = np.dot(temp, d)
  Pc = np.identity(2*dim) - np.dot(temp,C)

  G = matrix(-np.identity(2*dim), tc='d')
  h = matrix(np.zeros(((2*dim,1))), tc='d')
  AA = matrix(C, tc='d')
  b = matrix(d, tc='d')

  def proj(z):
    P = matrix(np.identity(2*dim), tc='d')
    q = matrix(-z, tc='d')
    res = solvers.qp(P, q, G, h, AA, b)
    return res['x']

  np.random.seed(0)

  # Initialize players
  init = np.random.rand(2*dim, 1)
  init[:dim] = init[:dim] / np.sum(init[:dim]) # ensuring it is part of the simplex
  init[dim:] = init[dim:] / np.sum(init[dim:])  

  finished, cnt, t0 = False, 0, time.time()

  # # EG ------------
  z = np.copy(init)
  for _ in range(T):
    cnt += 1
    zz = proj(z-lr*np.dot(A,z))
    z = proj(z-lr*np.dot(A,zz))
    if np.linalg.norm(z-x_opt)/np.linalg.norm(x_opt) <= eps or time.time()-t0 > max_time:
      print(f"Reached a relative error of {eps} after {cnt} iterations in {time.time()-t0:.2f} sec.")
      break


# Experiment on iterations to reach a fixed relative error

In [ ]:
dim = 500  # dim(x1) == dim(x2) == dim
x_opt = np.ones((2*dim,1))/dim
_seed = 0
max_iters = 300

temp1 = np.concatenate((np.ones((1,dim)), np.zeros((1,dim))),axis=1)
temp2 = np.concatenate((np.zeros((1,dim)), np.ones((1,dim))),axis=1)
C = np.concatenate((temp1,temp2), axis=0)
d = np.ones((2,1))
temp = np.linalg.inv(np.dot(C, C.T))
temp = np.dot(C.T, temp)
dc = np.dot(temp, d)
Pc = np.identity(2*dim) - np.dot(temp,C)

G = matrix(-np.identity(2*dim), tc='d')
h = matrix(np.zeros(((2*dim,1))), tc='d')
AA = matrix(C, tc='d')
b = matrix(d, tc='d')

def f_y(y,c,mu): 
  """ Update y in Alg. ACVI. """
  return -mu * np.sum(np.log(y)) + beta/2 * np.sum((y-c)**2)

def proj(z):
  P = matrix(np.identity(2*dim), tc='d')
  q = matrix(-z, tc='d')
  res = solvers.qp(P, q, G, h, AA, b)
  return res['x']

np.random.seed(_seed)
init = np.random.rand(2*dim, 1)
init[:dim] = init[:dim] / np.sum(init[:dim])
init[dim:] = init[dim:] / np.sum(init[dim:])

eps = .02
gamma = .3  # step-size for EG, OGDA, LA
num_acvi, num_i_acvi, num_gda, num_eg, num_la, num_ogda = [], [], [], [], [], []  
rot = np.linspace(.01, .99, 30) 

for a in rot:
  A1 = np.concatenate((a*np.identity(dim), (1-a)*np.identity(dim)), axis=1)
  A2 = np.concatenate((-(1-a)*np.identity(dim), a*np.identity(dim)), axis=1)
  A = np.concatenate((A1, A2), axis=0)

  # Inexact ACVI ------------
  beta, mu, delta, K, gamma_, T = .5, 1e-6, .8, 10, 0.05, 100
  y, lmd = init, np.zeros(((2*dim,1)))
  finished, cnt = False, 0
  z_x = np.copy(init)
  z_y = np.copy(init)
  z_lmd = np.zeros(init.shape)
  for t in range(T):
    mu *= delta
    flag = 1
    for k in range(K):
      cnt += 1
      # Solve approximately the X problem
      for _ in range(10):
        g = z_x + 1/beta * np.dot(Pc, np.dot(A,z_x)) - np.dot(Pc, z_y) + 1/beta * np.dot(Pc, z_lmd) - dc
        z_x -= gamma_ * g

      if np.linalg.norm(z_x-x_opt)/np.linalg.norm(x_opt) <= eps or cnt == max_iters:
        finished = True
        num_i_acvi.append(cnt)
        break

      # Solve approximately the Y problem
      for _ in range(10):
        assert all(z_y > 0)
        g = - mu * 1/z_y + beta*(z_y - z_x - z_lmd/beta)
        z_y -= gamma_ * g
        
      # Update lambdas
      z_lmd += beta*(z_x-z_y)

    if flag == 0 or finished:
      break

  # ACVI ------------
  beta, mu, delta, T, K = .5, 1e-6, .5, 10, 1
  y, lmd = init, np.zeros(((2*dim,1)))
  finished, cnt = False, 0
  for t in range(T):
    mu *= delta
    flag = 1
    for k in range(K if t+1 < T else 290):
      cnt += 1
      x = linalg.solve(np.identity(2*dim) + np.dot(Pc,A)/beta,
                       np.dot(Pc, y-lmd/beta) + dc)
      if np.linalg.norm(x-x_opt)/np.linalg.norm(x_opt) <= eps or \
         cnt == max_iters:
        finished = True
        num_acvi.append(cnt)
        break
      res = opt.minimize(lambda z: f_y(z,x+lmd/beta,mu), y, method='SLSQP')
      if not res.success:
        print('Failed to update y. t={}. k={}'.format(t, k))
        flag = 0
        break
      y = res.x.reshape(-1,1)
      lmd += beta*(x-y)
    if flag == 0 or finished:
      break

  # GDA ------------
  z = np.copy(init)
  for t in range(max_iters):
    z = proj(z - gamma * np.dot(A,z))
    if np.linalg.norm(z-x_opt)/np.linalg.norm(x_opt) <= eps or t+1 >= max_iters:
      num_gda.append(t+1)
      break

  # EG ------------
  z = np.copy(init)
  for t in range(max_iters):
    zz = proj(z-gamma*np.dot(A,z))
    z = proj(z-gamma*np.dot(A,zz))
    if np.linalg.norm(z-x_opt)/np.linalg.norm(x_opt) <= eps or \
       (t+1)*2 >= max_iters:
      num_eg.append(2*(t+1))
      break

  # OGDA ------------
  z0 = np.copy(init)
  z1 = np.copy(init)
  for t in range(max_iters):
    z = proj(z1 - 2*gamma*np.dot(A,z1) + gamma*np.dot(A,z0))
    z0 = np.copy(z1)
    z1 = np.copy(z)
    if np.linalg.norm(z-x_opt) / np.linalg.norm(x_opt) <= eps or \
       t+1 >= max_iters:
      num_ogda.append(t+1)
      break

  # LA ------------
  z = np.copy(init)
  la_k, la_alpha = 5, 0.5
  for t in range(max_iters):
    z_old = np.copy(z)
    for _ in range(la_k):
      #z = z - gamma * np.dot(A,z)
      z = proj( z - gamma * np.dot(A, z))
    z = proj(la_alpha * z_old + (1-la_alpha) * z)
    if np.linalg.norm(z - x_opt) / np.linalg.norm(x_opt) <= eps or \
       (t+1) * la_k >= max_iters:
      num_la.append(min(la_k*(t+1), max_iters))
      break

# Experiment on CPU-time for varying relative error

In [ ]:
dim = 500  # dim(x1) == dim(x2) == dim
x_opt = np.ones((2*dim,1))/dim
_seed = 0
max_iters = 300

temp1 = np.concatenate((np.ones((1,dim)), np.zeros((1,dim))),axis=1)
temp2 = np.concatenate((np.zeros((1,dim)), np.ones((1,dim))),axis=1)
C = np.concatenate((temp1,temp2), axis=0)
d = np.ones((2,1))
temp = np.linalg.inv(np.dot(C, C.T))
temp = np.dot(C.T, temp)
dc = np.dot(temp, d)
Pc = np.identity(2*dim) - np.dot(temp,C)

G = matrix(-np.identity(2*dim), tc='d')
h = matrix(np.zeros(((2*dim,1))), tc='d')
AA = matrix(C, tc='d')
b = matrix(d, tc='d')

def f_y(y,c,mu): 
  """ Update y in Alg. ACVI. """
  return -mu * np.sum(np.log(y)) + beta/2 * np.sum((y-c)**2)

def proj(z):
  P = matrix(np.identity(2*dim), tc='d')
  q = matrix(-z, tc='d')
  res = solvers.qp(P, q, G, h, AA, b)
  return res['x']

np.random.seed(_seed)
init = np.random.rand(2*dim, 1)
init[:dim] = init[:dim] / np.sum(init[:dim])
init[dim:] = init[dim:] / np.sum(init[dim:])

aa = .05
max_cpu_time = 1500  # in sec
max_iters = 3000
la_k, la_alpha = 5, .5  # hyperparameters for LA-GDA 
gamma = .1  # Step-size for GDA, EG, OGDA, LA-GDA
A1 = np.concatenate((aa*np.identity(dim),(1-aa)*np.identity(dim)),axis=1)
A2 = np.concatenate((-(1-aa)*np.identity(dim),aa*np.identity(dim)),axis=1)
A = np.concatenate((A1,A2), axis=0)
t_acvi, t_i_acvi, t_gda, t_eg, t_ogda, t_lak = [], [], [], [], [], []
eps = [.5, .1, .05, .01, .001, .0005]

for de in eps:

  # Inexact ACVI ------------
  beta, mu, delta, K, gamma_, T = .5, 1e-6, .8, 10, 0.05, 100
  y, lmd = init, np.zeros(((2*dim,1)))
  finished, cnt = False, 0
  z_x = np.copy(init)
  z_y = np.copy(init)
  z_lmd = np.zeros(init.shape)
  t0 = time.time()
  for t in range(T):
    mu *= delta
    for k in range(K):
      cnt += 1
      # Solve approximately the X problem
      for _ in range(10):
        g = z_x + 1/beta * np.dot(Pc, np.dot(A,z_x)) - np.dot(Pc, z_y) + 1/beta * np.dot(Pc, z_lmd) - dc
        z_x -= gamma_ * g
      if np.linalg.norm(z_x-x_opt)/np.linalg.norm(x_opt) <= de or time.time()-t0 >= max_cpu_time:
        finished = True
        t_i_acvi.append(time.time()-t0)
        break
      # Solve approximately the Y problem
      for _ in range(10):
        assert all(z_y >= 0)
        g = - mu * 1/z_y + beta*(z_y - z_x - z_lmd/beta)
        z_y -= gamma_ * g
      # Update lambdas
      z_lmd += beta*(z_x-z_y)
    if finished:
      break

  # ACVI ------------
  beta, mu, delta, T, K = .5, 1e-6, .5, 10, 1
  y, lmd = init,np.zeros(((2*dim,1)))
  t0 = time.time()
  for t in range(T):
    mu *= delta
    flag = 1
    for k in range(K if t+1 < T else 290):
      x = linalg.solve(np.identity(2*dim)+np.dot(Pc,A)/beta,np.dot(Pc,y-lmd/beta)+dc)
      if np.linalg.norm(x-x_opt)/np.linalg.norm(x_opt) <= de or time.time()-t0 >= max_cpu_time:
        flag = 0
        t_acvi.append(time.time()-t0)
        break
      res = opt.minimize(lambda z: f_y(z,x+lmd/beta,mu), y, method='SLSQP', jac=None)
      if not res.success:
        print('Failed to update y. t={}. k={}'.format(t,k))
        flag = 0
        break
      y = res.x.reshape(-1,1)
      lmd += beta*(x-y)
    if flag == 0:
      break

  # GDA ------------
  z = np.copy(init)
  t0 = time.time()
  for t in range(max_iters):
    z = proj(z-gamma*np.dot(A,z))
    if np.linalg.norm(z-x_opt) / np.linalg.norm(x_opt) <= de or \
        time.time()-t0 >= max_cpu_time:
      t_gda.append(time.time()-t0)
      break

  # EG ------------
  z = np.copy(init)
  t0 = time.time()
  for t in range(max_iters):
    zz = proj(z-gamma*np.dot(A,z))
    z = proj(z-gamma*np.dot(A,zz))
    if np.linalg.norm(z-x_opt)/np.linalg.norm(x_opt) <= de or \
        time.time()-t0 >= max_cpu_time:
      t_eg.append(time.time()-t0)
      break
  # OGDA ------------
  z0 = np.copy(init)
  z1 = np.copy(init)
  t0 = time.time()
  for t in range(max_iters):
    z = proj(z1 - 2*gamma*np.dot(A,z1) + gamma*np.dot(A,z0))
    z0 = np.copy(z1)
    z1 = np.copy(z)
    if np.linalg.norm(z-x_opt)/np.linalg.norm(x_opt) <= de or \
        time.time()-t0 >= max_cpu_time:
      t_ogda.append(time.time()-t0)
      break

  # LAK-GDA ------------
  la_k, la_alpha = 5, .5 
  z = np.copy(init)
  t0 = time.time()
  for t in range(max_iters):
    z_old = z
    for i in range(la_k):
      z = proj(z-gamma*np.dot(A, z))
    z = proj(la_alpha*z_old + (1-la_alpha)*z)
    if np.linalg.norm(z-x_opt) / np.linalg.norm(x_opt) <= de or \
        time.time()-t0 >= max_cpu_time:
      t_lak.append(time.time()-t0)
      break

# Experiment on warm-up

In [ ]:
from numpy.core.arrayprint import format_float_scientific
import time

eps = 1e-5
max_iters = 3000
a = 0.05

A1 = np.concatenate((a*np.identity(dim), (1-a)*np.identity(dim)), axis=1)
A2 = np.concatenate((-(1-a)*np.identity(dim), a*np.identity(dim)), axis=1)
A = np.concatenate((A1, A2), axis=0)

warmup = [5,10,20,30,40,50,60,70,80,90,100,110,120,130]

num_grads = []
all_times = []

for K_ in [1,5,10,20,30,40,50,60,70]:

  num_grad = []
  t_i_acvi = []

  for K_0 in warmup:

    print(f"K={K_}, K0={K_0}")

    # Inexact ACVI ------------
    beta, mu, delta, gamma_, T_ = .5, 1e-6, .8, 0.05, 5000
    n_grads = 0
    finished, cnt = False, 0
    z_x = np.copy(init)
    z_y = np.copy(init)
    z_lmd = np.zeros(init.shape)
    t0 = time.time()
    for t in range(0, T_):
      mu *= delta
      #print(mu)
      for k in range(0, K_0 if t == 0 else K_): # if t+1 < T_ else last_K):
        cnt += 1
        # Solve approximately the X problem
        for _ in range(10): 
          n_grads += 1
          g = z_x + 1/beta * np.dot(Pc, np.dot(A,z_x)) - np.dot(Pc, z_y) + 1/beta * np.dot(Pc, z_lmd) - dc
          z_x -= gamma_ * g

        if np.linalg.norm(z_x-x_opt)/np.linalg.norm(x_opt) <= eps or cnt == max_iters:
          finished = True
          num_grad.append(n_grads)
          t_i_acvi.append(time.time()-t0)
          print(f"num grad to reach distance of 1e-5: {n_grads}, time: {t_i_acvi[-1]}s")
          break

        # Solve approximately the Y problem
        for _ in range(10):
          n_grads += 1
          assert all(z_y > 0)
          g = - mu * 1/z_y + beta*(z_y - z_x - z_lmd/beta)
          z_y -= gamma_ * g
          
        # Update lambdas
        z_lmd += beta*(z_x-z_y)

      if finished:
        break

  num_grads.append(num_grad)
  all_times.append(t_i_acvi)

print(num_grads)